In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer

In [108]:
def get_probabilty(model, row):
    ''' Функция возвращает по найденному уравнению разделяющей плоскости вероятность объекта быть идеальным
    x - это признаки данного объекта
    b - вектор коэффициентов в уравнении плоскости '''
    des = model.decision_function([row])
    return 1 / (1 + np.exp(-des[0]))

Делим исходные данные на признаки и ответы, а также разделяем на данные, на которых обучаем и на которых тестируем

In [109]:
df = pd.read_csv('../assets/train.csv')

df.loc[df['result'] == 0.0, 'result'] = -1.0

X = df.loc[:, :'pollutedPointsPercentage']
y = df.loc[:, 'result']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
copy_of_x_test = x_test.copy()

Стандартизация данных

In [110]:
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)
print(x_test)

[[ 0.48464326  1.43040215  0.17287295 -1.03557979 -1.01151904]
 [-1.04384142 -0.99438479 -0.707756    0.37169514  0.98102328]
 [ 1.2454585   1.9128533   0.79960918 -1.14540908 -0.84673438]
 ...
 [-1.78409409 -1.08836878 -1.21204143  0.06010762  1.36826064]
 [-0.94788274 -0.93799439 -0.92338706  1.91147202  0.54116824]
 [-1.73611475 -0.9317288  -1.06437358  1.70208293  1.26976935]]


Объявление и обучение модели

In [111]:
model = svm.SVC(kernel='linear', C = 1.0, probability=True)
model.fit(x_train, y_train)

SVC(kernel='linear', probability=True)

Применение модели

In [112]:
probabilities = np.array([])
x_test = scaler.transform(np.array([[55.77658176583015, 111.5531635316603, 129439.50481135711, 500, 0.05]]))
# x_test = np.array(x_test)

for x in x_test:
   prob = get_probabilty(model, list(x)) * 100
   probabilities = np.append(probabilities, prob)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [113]:
np.savetxt("../assets/final_result.csv", probabilities, delimiter=",", fmt='%10.5f')
# "areasAmountPer100k":106.48354666492484
# sportsObjectsAmountPer100k":122.2588
# areasSquarePer100k":567272.50481135711
# dist: 500
# 0.05


Экспортирование модели в папку `/models`

In [114]:
import joblib
joblib.dump(scaler, '../models/scaler.pkl')
joblib.dump(model, '../models/model.pkl')

['../models/model.pkl']